In [3]:
import feedparser
import pandas as pd
from urllib.parse import quote
from bs4 import BeautifulSoup
import requests
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


pd.options.display.max_columns = 250    #Changes the number of columns diplayed (default is 20)
pd.options.display.max_rows = 250       #Changes the number of rows diplayed (default is 60)
pd.options.display.max_colwidth = 250   #Changes the number of characters in a cell so that the contents don't get truncated (default is 50)

In [32]:
# Source - https://news.google.com/
# XML - https://news.google.com/rss?hl=cs&gl=CZ&ceid=CZ:cs
# Query - https://stackoverflow.com/questions/51537063/url-format-for-google-news-rss-feed

In [4]:
def analyze_sentiment(text):
    analyzer = SentimentIntensityAnalyzer()
    sentiment = analyzer.polarity_scores(text)
    return sentiment['compound']

In [22]:
import feedparser
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import quote

def parseRSS(rss_url):
    return feedparser.parse(rss_url)

def getContent(rss_url, query):
    dates = []
    sources = []
    headlines = []
    headline_sentiments = []
    links = []

    feed = parseRSS(rss_url)
    for item in feed.entries:
        if query.lower() in item.title.lower():
            # Získání data publikace
            published_date = pd.to_datetime(item.published)
            # Filtrování zpráv podle roku 2021
            if published_date.year >= 2021:
                dates.append(item.published)
                sources.append(item.source.title)
                headlines.append(item.title)
                headline_sentiments.append(analyze_sentiment(item.title))  # Analyze sentiment of the headline
                # Descriptions.append(get_article_content(item.link))
                links.append(item.link)

    return pd.DataFrame({'Date': dates, 'Source': sources, 'Title': headlines, 'Link': links, 'Sentiment': headline_sentiments}) #, 'Description': descriptions})

def get_article_content(article_url):
    response = requests.get(article_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    article_content = soup.get_text()
    return article_content

query = "Schillerová"
encoded_query = quote(query)  # Encoding the search term
lang = {
    'cz': 'hl=cs&gl=CZ&ceid=CZ:cs',
    'en': 'hl=en&gl=US&ceid=US:en'
}

rss_url = f"https://news.google.com/rss/search?q={encoded_query}&{lang['cz']}"

df = getContent(rss_url, query)

display(df)


NameError: name 'Descriptions' is not defined